In [ ]:
%pip install -qU mem0ai azure-identity langchain langchain_voyageai pymongo

In [ ]:
import os

# 使用 Azure 时加上这行，否则会走到 Azure 的 tracing 系统，报401Error
os.environ["OPENAI_AGENTS_DISABLE_TRACING"] = "true"

# 导入 Azure OpenAI Python 客户端库
from openai import AzureOpenAI

# 初始化Azure OpenAI 客户端
# 将使用环境变量 OPENAI_API_KEY 中的 API 密钥
#openai_client = OpenAI()
openai_client = AzureOpenAI(
    api_version="2025-03-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_deployment="gpt-4o",
)


In [ ]:
from langchain_voyageai import VoyageAIEmbeddings


config = {
    "vector_store": {
        "provider": "mongodb",
        "config": {
            "db_name": "mem0_agent_memory_test",
            "collection_name": "extracted_memories",
            "mongo_uri": os.environ["MONGODB_URI"],
            "embedding_model_dims": 1024
        }
    },
    "embedder": {
        "provider": "langchain",
        "config": {
            "model": VoyageAIEmbeddings(
                model="voyage-4-large",
                voyage_api_key=os.getenv("VOYAGE_API_KEY")
            ),
            "embedding_dims": 1024
        }
    },
    "llm": {
        "provider": "azure_openai",
        "config": {
            "model": "gpt-4o",
            "azure_kwargs": {
                "api_key": os.getenv("OPENAI_API_KEY"),
                "azure_endpoint": os.getenv("AZURE_OPENAI_ENDPOINT"),
                "azure_deployment": "gpt-4o",
                "api_version": "2025-03-01-preview"
            }
        }
    }
}

In [ ]:
# 常量
CURRENT_USER_ID = "use001"

In [ ]:
from mem0 import Memory
memory = Memory.from_config(config)

In [ ]:
messages = [
    {"role": "user", "content": "Hi, I'm Alex. I love basketball and gaming."},
    {"role": "assistant", "content": "Hey Alex! I'll remember your interests."}
]
memory.add(messages, user_id=CURRENT_USER_ID)

# m.search(query="What does Alex like?", user_id=CURRENT_USER_ID)
# m.delete(user_id=CURRENT_USER_ID)

In [ ]:
results = memory.search("What do you know about me?", user_id=CURRENT_USER_ID)
print(results)